In [ ]:
!pip install transformers
!pip install bert-score
!pip install datasets
!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 58.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 91.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 28.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 3.1 MB/s eta 0:00:00
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3021, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2815, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/l

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

MessageError: ignored

In [ ]:
import torch
print(torch.cuda.get_device_name(0))
SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

Tesla T4


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# Set the path to your JSONL file
training_file = "/content/gdrive/MyDrive/cse_635/train.jsonl"
validation_file="/content/gdrive/MyDrive/cse_635/validation.jsonl"

In [ ]:
def getanswers(strs ,position):
    nums=position[0][0]
    start=0
    for i in range(len(strs)):
      if nums==i:
        start+=position[0][1]
        break
      start+=len(strs[i])
    
    return ''.join(strs),start

In [ ]:
import json
from torch.utils.data import Dataset, DataLoader

class MyDataset(Dataset):
    def __init__(self, filepath, type):
        self.data = []
        self.max_length=0
        self.id=0
        with open(filepath, 'r') as f:
            for line in f:
                json_obj = json.loads(line)
                if json_obj['tags'][0]==type:
                    question = json_obj['postText'][0]
                    context = json_obj['targetParagraphs']
                    answer = json_obj['spoiler']
                    strs,start=getanswers(context,json_obj['spoilerPositions'][0])
                    context=strs
                    extracted_data = {
                        'answers': {'answer_start': [start], 'text': answer},
                        'context': context,
                        'question': question,
                        'id':str(self.id),
                    }
                    self.data.append(extracted_data)
                    self.max_length=max(self.max_length,len(context)+len(question[0]))
                self.id+=1

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

In [ ]:
# Load the dataset using DataLoader
training = MyDataset(training_file,'phrase')
validation = MyDataset(validation_file,'phrase')

NameError: ignored

In [ ]:
print(training[0])
print(validation[0])

{'answers': {'answer_start': [0], 'text': ['2070']}, 'context': '2070 is shaping up to be a great year for Mother Earth.That\'s when NASA scientists are predicting the hole in the ozone layer might finally make a full recovery. Researchers announced their conclusion, in addition to other findings, in a presentation Wednesday during the annual American Geophysical Union meeting in San Francisco.The team of scientists specifically looked at the chemical composition of the ozone hole, which has shifted in both size and depth since the passing of the Montreal Protocol in 1987. The agreement banned its 197 signatory countries from using chemicals, like chlorofluorocarbons (CFCs), that break down into chlorine in the upper atmosphere and harm the ozone layer.They found that, while levels of chlorine in the atmosphere have indeed decreased as a result of the protocol, it\'s too soon to tie them to a healthier ozone layer."Ozone holes with smaller areas and a larger total amount of ozone are n

In [ ]:

from transformers import RobertaTokenizer, RobertaModel
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

In [ ]:
from datasets import Dataset
dset_training = Dataset.from_list(training)
dset_training=dset_training.with_format("torch")

dset_validation = Dataset.from_list(validation)
dset_validation=dset_validation.with_format("torch")

In [ ]:
context = dset_training[0]["context"]
question = dset_training[0]["question"]

inputs = tokenizer(question, context)
tokenizer.decode(inputs["input_ids"])

'<s>NASA sets date for full recovery of ozone hole</s></s>2070 is shaping up to be a great year for Mother Earth.That\'s when NASA scientists are predicting the hole in the ozone layer might finally make a full recovery. Researchers announced their conclusion, in addition to other findings, in a presentation Wednesday during the annual American Geophysical Union meeting in San Francisco.The team of scientists specifically looked at the chemical composition of the ozone hole, which has shifted in both size and depth since the passing of the Montreal Protocol in 1987. The agreement banned its 197 signatory countries from using chemicals, like chlorofluorocarbons (CFCs), that break down into chlorine in the upper atmosphere and harm the ozone layer.They found that, while levels of chlorine in the atmosphere have indeed decreased as a result of the protocol, it\'s too soon to tie them to a healthier ozone layer."Ozone holes with smaller areas and a larger total amount of ozone are not nece

In [ ]:
max_length = 384
stride = 128

def preprocess_function_training(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
        
    )

    offset_mapping = inputs.pop("offset_mapping")
    sample_map = inputs.pop("overflow_to_sample_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        sample_idx = sample_map[i]
        answer = answers[sample_idx]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label is (0, 0)
        if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [ ]:
tokenized_training = dset_training.map(preprocess_function_training, batched=True,remove_columns=dset_training.column_names)

Map:   0%|          | 0/1367 [00:00<?, ? examples/s]

In [ ]:
def preprocess_function_validation(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_map = inputs.pop("overflow_to_sample_mapping")
    example_ids = []

    for i in range(len(inputs["input_ids"])):
        sample_idx = sample_map[i]
        example_ids.append(examples["id"][sample_idx])

        sequence_ids = inputs.sequence_ids(i)
        offset = inputs["offset_mapping"][i]
        inputs["offset_mapping"][i] = [
            o if sequence_ids[k] == 1 else None for k, o in enumerate(offset)
        ]

    inputs["example_id"] = example_ids

    offset_mapping = inputs["offset_mapping"]
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        sample_idx = sample_map[i]
        answer = answers[sample_idx]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label is (0, 0)
        if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions

    return inputs

In [ ]:
tokenized_validation = dset_validation.map(preprocess_function_validation, batched=True,remove_columns=dset_validation.column_names)

Map:   0%|          | 0/335 [00:00<?, ? examples/s]

In [ ]:
from transformers import RobertaForQuestionAnswering

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model = RobertaForQuestionAnswering.from_pretrained('roberta-base').to(device)
output_dir="/content/gdrive/MyDrive/cse_635/individual_project/code/baseline/roberta_phrase"


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForQuestionAnswering: ['lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForQuestionAnswering were not initialized from the model checkpoint at roberta-base and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to use 

In [ ]:
import evaluate

bleu = evaluate.load('bleu')
meteor = evaluate.load('meteor')
bertscore=evaluate.load('bertscore')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [ ]:
from tqdm.auto import tqdm
import collections
import numpy as np
n_best = 20
max_answer_length = 30

def compute_metrics(start_logits, end_logits, features, examples):
    example_to_features = collections.defaultdict(list)
    for idx, feature in enumerate(features):
        example_to_features[feature["example_id"]].append(idx)
    predicted_answers = []
    for example in tqdm(examples):
        example_id = example["id"]
        context = example["context"]
        answers = []
        # Loop through all features associated with that example
        for feature_index in example_to_features[example_id]:
            start_logit = start_logits[feature_index]
            end_logit = end_logits[feature_index]
            offsets = features[feature_index]["offset_mapping"]

            start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
            end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Skip answers that are not fully in the context
                    if offsets[start_index] is None or offsets[end_index] is None:
                        continue
                    # Skip answers with a length that is either < 0 or > max_answer_length
                    if (
                        end_index < start_index
                        or end_index - start_index + 1 > max_answer_length
                    ):
                        continue
                    answer = {
                        "text": context[offsets[start_index][0] : offsets[end_index][1]],
                        "logit_score": start_logit[start_index] + end_logit[end_index],
                    }
                    answers.append(answer)
        # Select the answer with the best score
        if len(answers) > 0:
            best_answer = max(answers, key=lambda x: x["logit_score"])
            predicted_answers.append(best_answer["text"])
        else:
            predicted_answers.append("")
    
    print(predicted_answers[0:20])
    theoretical_answers = [ex["answers"]["text"][0] for ex in examples]
    print(theoretical_answers[0:20])
    bleu_score=bleu.compute(predictions=predicted_answers, references=theoretical_answers)
    meteor_score=meteor.compute(predictions=predicted_answers, references=theoretical_answers)
    bertscore_score=bertscore.compute(predictions=predicted_answers, references=theoretical_answers,lang="en",model_type="distilbert-base-uncased")
    return bleu_score,meteor_score,bertscore_score

In [ ]:
from transformers import TrainingArguments

args = TrainingArguments(
    output_dir=output_dir,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="epoch",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    learning_rate=2e-5,
    num_train_epochs=6,
    weight_decay=0.01,
    fp16=True,
)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_training,
    eval_dataset=tokenized_validation,
    tokenizer=tokenizer,
)
trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.345200,1.776442
2,0.197000,1.930246
3,0.107600,2.180506
4,0.078200,2.247855
5,0.065900,2.199953
6,0.063300,2.223924


TrainOutput(global_step=1182, training_loss=0.14287789501312823, metrics={'train_runtime': 578.4697, 'train_samples_per_second': 32.621, 'train_steps_per_second': 2.043, 'total_flos': 3698002349706240.0, 'train_loss': 0.14287789501312823, 'epoch': 6.0})

In [ ]:
best_checkpoint = RobertaForQuestionAnswering.from_pretrained("/content/gdrive/MyDrive/cse_635/individual_project/code/baseline/roberta_phrase/checkpoint-985")
checkpoint_tokenizer=RobertaTokenizer.from_pretrained("/content/gdrive/MyDrive/cse_635/individual_project/code/baseline/roberta_phrase/checkpoint-985")
trainer_best_checkpoint = Trainer(
    model=best_checkpoint,
    args=args,
    train_dataset=tokenized_training,
    eval_dataset=tokenized_validation,
    tokenizer=checkpoint_tokenizer,
)

In [ ]:
predictions, _, _ = trainer_best_checkpoint.predict(tokenized_validation)
start_logits, end_logits = predictions
bl, me, be= compute_metrics(start_logits, end_logits, tokenized_validation, dset_validation)
print(bl)
print(me)
print("bert score: {precision: "+ str(be['precision'][0])+" recall: "+str(be['recall'][0])+" f1: "+str(be['f1'][0])+"}")

  0%|          | 0/335 [00:00<?, ?it/s]

['$3 to $5', 'Sprite', 'Garbanzo Beans', 'Anthony Bourdain', 'naled', 'don’t fart', 'earlier', 'more is known', 'Mike Rounds', 'Jalapeno Popper Quesarito', 'Edward Gorey', 'Rag & Bone', 'pixie cut', 'southern flying squirrel', 'antique cabinet', 'August 6th', "we don't know for sure", '"but"', 'Arizona Republic', '32 percent of them said yes']
['20%', 'Sprite', 'Smoky Paprika-Baked Garbanzo Beans', 'Anthony Bourdain', 'Dibrom', 'They don’t fart', 'starts later', 'bricking iPad Pros', 'Stace Nelson', 'reduced fat sour cream', 'Edward Gorey', 'Rag & Bone', 'pixie cut', 'southern flying squirrel', "Hope's antique cabinet", 'August 6th', 'perfectly average', '"but"', 'The Arizona Republic', 'apocalyptic omen']
{'bleu': 0.28489320664792833, 'precisions': [0.5252631578947369, 0.3967479674796748, 0.23249299719887956, 0.13596491228070176], 'brevity_penalty': 1.0, 'length_ratio': 1.1124121779859484, 'translation_length': 950, 'reference_length': 854}
{'meteor': 0.5169815987931801}
bert score: {

In [ ]:
best_checkpoint = RobertaForQuestionAnswering.from_pretrained("/content/gdrive/MyDrive/cse_635/individual_project/code/baseline/roberta_phrase/checkpoint-788")
checkpoint_tokenizer=RobertaTokenizer.from_pretrained("/content/gdrive/MyDrive/cse_635/individual_project/code/baseline/roberta_phrase/checkpoint-788")
trainer_best_checkpoint = Trainer(
    model=best_checkpoint,
    args=args,
    train_dataset=tokenized_training,
    eval_dataset=tokenized_validation,
    tokenizer=checkpoint_tokenizer,
)

In [ ]:
predictions, _, _ = trainer_best_checkpoint.predict(tokenized_validation)
start_logits, end_logits = predictions
bl, me, be= compute_metrics(start_logits, end_logits, tokenized_validation, dset_validation)
print(bl)
print(me)
print("bert score: {precision: "+ str(be['precision'][0])+" recall: "+str(be['recall'][0])+" f1: "+str(be['f1'][0])+"}")

  0%|          | 0/335 [00:00<?, ?it/s]

['$3 to $5', 'Sprite', 'Smoky Paprika-Baked Garbanzo Beans', 'Anthony Bourdain', 'naled', 'They don’t fart', 'earlier', 'more is known', 'Mike Rounds', 'reduced fat sour cream', 'Edward Gorey', 'Rag & Bone', 'pixie cut', 'southern flying squirrel', 'antique cabinet', 'August 6th', "we don't know for sure", 'and', 'Arizona Republic', '32 percent of them said yes']
['20%', 'Sprite', 'Smoky Paprika-Baked Garbanzo Beans', 'Anthony Bourdain', 'Dibrom', 'They don’t fart', 'starts later', 'bricking iPad Pros', 'Stace Nelson', 'reduced fat sour cream', 'Edward Gorey', 'Rag & Bone', 'pixie cut', 'southern flying squirrel', "Hope's antique cabinet", 'August 6th', 'perfectly average', '"but"', 'The Arizona Republic', 'apocalyptic omen']
{'bleu': 0.33327494569772026, 'precisions': [0.5566343042071198, 0.4375, 0.2852941176470588, 0.17757009345794392], 'brevity_penalty': 1.0, 'length_ratio': 1.085480093676815, 'translation_length': 927, 'reference_length': 854}
{'meteor': 0.5336928651792094}
bert sc

In [ ]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator()

In [ ]:
from transformers import RobertaForQuestionAnswering

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model = RobertaForQuestionAnswering.from_pretrained('bert-base-cased').to(device)
output_dir="/content/gdrive/MyDrive/cse_635/individual_project/code/baseline/roberta_phrase"

You are using a model of type bert to instantiate a model of type roberta. This is not supported for all configurations of models and can yield errors.


Some weights of the model checkpoint at bert-base-cased were not used when initializing RobertaForQuestionAnswering: ['bert.encoder.layer.6.output.LayerNorm.weight', 'bert.encoder.layer.0.intermediate.dense.bias', 'bert.encoder.layer.3.attention.self.query.bias', 'bert.encoder.layer.11.output.LayerNorm.weight', 'bert.encoder.layer.10.attention.self.value.bias', 'bert.encoder.layer.5.attention.output.dense.bias', 'bert.encoder.layer.5.attention.output.dense.weight', 'bert.encoder.layer.4.attention.self.value.bias', 'bert.encoder.layer.3.intermediate.dense.bias', 'bert.encoder.layer.11.attention.output.dense.weight', 'bert.encoder.layer.7.attention.output.LayerNorm.weight', 'bert.encoder.layer.3.intermediate.dense.weight', 'bert.encoder.layer.1.output.dense.bias', 'bert.encoder.layer.7.attention.self.value.bias', 'bert.encoder.layer.1.attention.self.query.bias', 'bert.encoder.layer.11.attention.output.LayerNorm.bias', 'bert.encoder.layer.1.attention.output.dense.weight', 'bert.encoder.la

In [ ]:
from torch.utils.data import DataLoader
from transformers import default_data_collator

tokenized_training.set_format("torch")
validation_set = tokenized_validation.remove_columns(["example_id", "offset_mapping"])
validation_set.set_format("torch")

train_dataloader = DataLoader(
    tokenized_training,
    shuffle=True,
    collate_fn=default_data_collator,
    batch_size=8,
)
eval_dataloader = DataLoader(
    validation_set, collate_fn=default_data_collator, batch_size=8
)

In [ ]:
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=2e-5)

In [ ]:
from accelerate import Accelerator

accelerator = Accelerator(mixed_precision="fp16")
model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader
)

In [ ]:
from transformers import get_scheduler

num_train_epochs = 3
num_update_steps_per_epoch = len(train_dataloader)
num_training_steps = num_train_epochs * num_update_steps_per_epoch

lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

In [ ]:
import evaluate

bleu = evaluate.load('bleu')
meteor = evaluate.load('meteor')
bertscore=evaluate.load('bertscore')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [ ]:
from tqdm.auto import tqdm
import collections
n_best = 20
max_answer_length = 30

def compute_metrics(start_logits, end_logits, features, examples):
    example_to_features = collections.defaultdict(list)
    for idx, feature in enumerate(features):
        example_to_features[feature["example_id"]].append(idx)
    predicted_answers = []
    for example in tqdm(examples):
        example_id = example["id"]
        context = example["context"]
        answers = []

        # Loop through all features associated with that example
        for feature_index in example_to_features[example_id]:
            start_logit = start_logits[feature_index]
            end_logit = end_logits[feature_index]
            offsets = features[feature_index]["offset_mapping"]

            start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
            end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Skip answers that are not fully in the context
                    if offsets[start_index] is None or offsets[end_index] is None:
                        continue
                    # Skip answers with a length that is either < 0 or > max_answer_length
                    if (
                        end_index < start_index
                        or end_index - start_index + 1 > max_answer_length
                    ):
                        continue
                    answer = {
                        "text": context[offsets[start_index][0] : offsets[end_index][1]],
                        "logit_score": start_logit[start_index] + end_logit[end_index],
                    }
                    answers.append(answer)
        # Select the answer with the best score
        if len(answers) > 0:
            best_answer = max(answers, key=lambda x: x["logit_score"])
            predicted_answers.append(best_answer["text"])
        else:
            predicted_answers.append("")
    
    print(predicted_answers[0:20])
    theoretical_answers = [ex["answers"]["text"][0] for ex in examples]
    print(theoretical_answers[0:20])
    bleu_score=bleu.compute(predictions=predicted_answers, references=theoretical_answers)
    meteor_score=meteor.compute(predictions=predicted_answers, references=theoretical_answers)
    bertscore_score=bertscore.compute(predictions=predicted_answers, references=theoretical_answers,lang="en",model_type="distilbert-base-uncased")
    return bleu_score,meteor_score,bertscore_score